In this lecture, we're going to talk about querying DataFrames. The first step in the process is to understand "Boolean Masking". A Boolean masking is the heart of fast and efficient querying and both NumPy and in Pandas, and it's analogous to "bit masking" which is used in other areas of computational science. By the end of this lecture, you'll understand how Boolean masking works and how to apply this to a DataFrame to get out the data that you're interested in.

Now, a Boolean mask is an array which can be thought of as a one dimension like a series, or two-dimensions like a DataFrame, where each of the values of the array are either true or false. This array is essentially overlaid on top of the other data structure that we're querying, and any cell aligned with the true value will be admitted into our final result, and any cell aligned with the false value will not. 



In [5]:
#Let's take a look. So let's start with an example and import our graduate admissions dataset. So first we'll bring in pandas:
import pandas as pd
#Then we'll load in our CSV data file:
df = pd.read_csv('datasets/Admission_Predict.csv', index_col=0)
#And then we're going to clean this up, and fix a couple of the poorly named columns like we did in previous lecture. So I'm
#going to use our list comprehension here x.lower.strip for x in df.columns. So these is just iterating over a bunch of strings,
#and turning them lower case, and stripping out whitespace from either side, 
df.columns = [x.lower().strip() for x in df.columns]
#And let's take a look at the results:
df.head()

,gre score,toefl score,university rating,sop,lor,cgpa,research,chance of admit
Serial No.,,,,,,,,
1,337,118,4,4.5,4.5,9.65,1,0.92
2,324,107,4,4.0,4.5,8.87,1,0.76
3,316,104,3,3.0,3.5,8.00,1,0.72
4,322,110,3,3.5,2.5,8.67,1,0.80
5,314,103,2,2.0,3.0,8.21,0,0.65


In [7]:
#So Boolean masks are created by applying operators directly to the pandas series or DataFrame object. For instance, in our
#graduate admission dataset, we might be interested in seeing only those students that have chance higher than 0.7 of being
#admitted. 

#To build a Boolean mask for this query, we want to project the chance of admit column using the indexing operator and apply
#the greater than operator with a comparison value of 0.7. This is essentially broadcasting a comparison operator, greater than,
#with the results being returned as a Boolean series. The resultant series is indexed where the value of each cell is either
#true or false depending on whether a student has a chance of admission higher than 0.7. 
admit_mask = df['chance of admit'] > 0.7
admit_mask
#So we'll create this thing called an admit mask, and we'll say df sub chance of admit, and then we're going to broadcast
#against this columns. Remember we're just projecting a single column. The greater than operator and pass into a value of 0.7,
#and let's take a look at what that mid-month mask it is. 

Serial No.
1       True
2       True
3       True
4       True
5      False
       ...  
396     True
397     True
398     True
399    False
400     True
Name: chance of admit, Length: 400, dtype: bool

#So this is pretty fundamentals. So take a moment to look at this. The result of broadcasting a comparison operator is a Boolean mask, true or false values depending on the results of the comparison. Underneath, pandas is applying the comparison operator that you specified through vectorization. So it's efficiently and in parallel to all of the values in the array that you specified, which in this case, is the chance of admit column of the DataFrame. The result is a series objects, since only one column is being operated on filled with either true or false values, which is what the comparison operator returns. 

In [8]:
#So what do you do with the Boolean mask once you formed it? Well, you could just lay it on top of the data and to hide the
#data that you don't want, which is represented by all the false values. We do this by using the .where() function on the
#original DataFrame:
df.where(admit_mask).head()
#So df.where, and we're just pass in a single parameter the admit mask, and let's take a look at what that DataFrame looks like,
#and we'll add the dot head.

,gre score,toefl score,university rating,sop,lor,cgpa,research,chance of admit
Serial No.,,,,,,,,
1,337.0,118.0,4.0,4.5,4.5,9.65,1.0,0.92
2,324.0,107.0,4.0,4.0,4.5,8.87,1.0,0.76
3,316.0,104.0,3.0,3.0,3.5,8.00,1.0,0.72
4,322.0,110.0,3.0,3.5,2.5,8.67,1.0,0.80
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
#So we see that the resulting DataFrame keeps the original index values, and only data which met the condition was retained. All
#of the rows which did not meet the condition have NaN data instead, but these rows weren't dropped from our dataset. They're
#still there. They're just not a number. 

#The next step is, of course, if we don't want the NaN data that we use dropna() function, and this is quite common:
df.where(admit_mask).dropna().head()


,gre score,toefl score,university rating,sop,lor,cgpa,research,chance of admit
Serial No.,,,,,,,,
1,337.0,118.0,4.0,4.5,4.5,9.65,1.0,0.92
2,324.0,107.0,4.0,4.0,4.5,8.87,1.0,0.76
3,316.0,104.0,3.0,3.0,3.5,8.00,1.0,0.72
4,322.0,110.0,3.0,3.5,2.5,8.67,1.0,0.80
6,330.0,115.0,5.0,4.5,3.0,9.34,1.0,0.90


In [10]:
#So the return data frame now has all of the NAN rows dropped. Notice that the index now includes one through four and six, but
#not five. 

#Despite being really handy, where() isn't actually used that often? Instead, the panda devs created a shorthand syntax which
#combines where() and dropna(), doing both at once, and in typical fashion, the just to overloaded the indexing operator to do
#this:
df[df['chance of admit'] > 0.7].head()
#So here's an example. So we do df, and then inside of the indexing operator, we actually put our computation for the Boolean
#mask. So df sub chance of admit, greater than 0.7.head. 

,gre score,toefl score,university rating,sop,lor,cgpa,research,chance of admit
Serial No.,,,,,,,,
1,337,118,4,4.5,4.5,9.65,1,0.92
2,324,107,4,4.0,4.5,8.87,1,0.76
3,316,104,3,3.0,3.5,8.00,1,0.72
4,322,110,3,3.5,2.5,8.67,1,0.80
6,330,115,5,4.5,3.0,9.34,1,0.90


In [11]:
#I personally find that much harder to read, but it's super common when you're reading other people's code. So it's important
#to be able to understand it and to write it. Just reviewing this indexing operator on the DataFrame, it now does three things:

#So first, it can be called with a string parameter to project a single column
df['gre score'].head()


Serial No.
1    337
2    324
3    316
4    322
5    314
Name: gre score, dtype: int64

In [13]:
#Or you can send it a list of columns as strings:
df[['gre score','toefl score']].head()


,gre score,toefl score
Serial No.,,
1,337,118
2,324,107
3,316,104
4,322,110
5,314,103


In [14]:
#Or you can send it a Boolean mask:
df[df['gre score'] > 320].head()


,gre score,toefl score,university rating,sop,lor,cgpa,research,chance of admit
Serial No.,,,,,,,,
1,337,118,4,4.5,4.5,9.65,1,0.92
2,324,107,4,4.0,4.5,8.87,1,0.76
4,322,110,3,3.5,2.5,8.67,1,0.80
6,330,115,5,4.5,3.0,9.34,1,0.90
7,321,109,3,3.0,4.0,8.20,1,0.75


#And each of these is mimicking functionality from either .loc(), or .where().dropna().

In [17]:
#Before we leave this, let's talk about combining multiple Boolean masks, such as multiple criteria for including. In bit
#asking and other places in computer science, this is done with "and". If both masks must be True, for a True value, and to be
#n the final mask, or "or" if only one needs to be True.

#Unfortunately, it doesn't feel quite as natural in pandas. For instance, if you want to take two Boolean series and add them
#ogether:
(df['chance of admit']>0.7) and (df['chance of admit']<0.9)


ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [18]:
#But that didn't work and despite using pandas for awhile, I actually find a regularly try and do what I just did. The problem is that
#you have a series object and Python underneath doesn't know how to compare two series using AND, or, Or. Instead, the pandas
#author have overwritten the pipe | and ampersand & operator to handle this for us:
(df['chance of admit']>0.7) & (df['chance of admit']<0.9)

Serial No.
1      False
2       True
3       True
4       True
5      False
       ...  
396     True
397     True
398    False
399    False
400    False
Name: chance of admit, Length: 400, dtype: bool

In [19]:
#So one other thing to watch out for is the order of operations and this gets me regularly as well. A common error for new
#pandas users is to try and do Boolean comparisons using the & operator, but not putting parentheses around the individual terms
#you are interested in:
df['chance of admit']>0.7 & df['chance of admit']<0.9


TypeError: Cannot perform 'rand_' with a dtyped [float64] array and scalar of type [bool]

In [20]:
#The problem is that Python is trying to bitwise and a 0.7, and a pandas DataFrame, when you really want a bitwise and broadcast
#DataFrames together.

#Another way to do this is actually just to get rid of the comparison operator completely, and instead to use the built-in
#functions which mimic this approach:
df['chance of admit'].gt(0.7) & df['chance of admit'].lt(0.9)


Serial No.
1      False
2       True
3       True
4       True
5      False
       ...  
396     True
397     True
398    False
399    False
400    False
Name: chance of admit, Length: 400, dtype: bool

In [21]:
#So these functions are built right into the series and DataFrame objects. So you can actually chain them too, which results in
#the same answer and the use of No visual operator. You can decide what looks best for you
df['chance of admit'].gt(0.7).lt(0.9)


Serial No.
1      False
2      False
3      False
4      False
5       True
       ...  
396    False
397    False
398    False
399     True
400    False
Name: chance of admit, Length: 400, dtype: bool

#So this only works if your operator, such as less than or greater than, is built into the DataFrame, but I certainly find the last code example to be much more readable than the one with the ampersands in the parentheses. So you might want to consider using this. 

#You need to be able to read and write all of these, and understand the implications of the route you're choosing. It's worth really going back and rewatching this lecture to make sure that you have it. I would say 50 percent or more of the work you'll be doing in data cleaning involves querying DataFrames. 



In this lecture, we've learned to query DataFrames using Boolean masking, which is extremely important and often used in the world of data science. With Boolean masking, we can select data based on the criteria we define, and frankly, you're going to use this everywhere. We've also seen how there are many different ways to query the DataFrame an interesting site implications that come up when doing so.